### **Problem Statement:**
Your task is to build a multi-layer **autoencoder** using **PyTorch** to compress the images into a lower-dimensional representation and then reconstruct them. The model should use the **Adam optimizer** and **Mean Squared Error (MSE) loss**. The goal is to minimize the reconstruction loss and generate an accurate representation of the input data.

#### **Steps to Complete the Exercise:**
1. **Build the Multi-Layer Autoencoder**:  
   - **Encoder**: Multiple dense layers reducing dimensionality.
   - **Decoder**: Symmetric layers reconstructing the image.
2. **Train the Model**: Use **Mean Squared Error (MSE) loss** and **Adam optimizer** to train the autoencoder.



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

the ANSI diagram for the multi-layer autoencoder
    Input Layer (28x28 pixels)  
          |
      [Flatten Layer] (784)
          |
      [Dense Layer 512] - Linear, ReLU
          |
      [Dense Layer 256] - Linear, ReLU
          |
      [Dense Layer 128] - Linear, ReLU
          |
      [Dense Layer 64]  <-- Latent Representation (Bottleneck)
          |
      [Dense Layer 128] - Linear, ReLU
          |
      [Dense Layer 256] - Linear, ReLU
          |
      [Dense Layer 512] - Linear, ReLU
          |
      [Dense Layer 784] - Sigmoid (Reconstructs 28x28 image)
          |
      [Unflatten Layer] (Reshape to 28x28)
          |
      Output (Reconstructed Image)



In [ ]:
### Implement the Autoencoder ###

# Define Autoencoder class
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784,512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,64)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(64,128),
            nn.ReLU(),
            nn.Linear(128,256),
            nn.ReLU(),
            nn.Linear(256,512),
            nn.ReLU(),
            nn.Linear(512,784),
            nn.Sigmoid(),
            nn.Unflatten(1,(28,28))
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded



In [ ]:
# Define loss function and optimizer
student_model = Autoencoder()
optimizer = optim.Adam(student_model.parameters())
# initialize the 'criterion' to apply the loss and 'optimizer' for the optimzation.
criterion = nn.MSELoss()

In [ ]:
# Autoencoder Training Function
def check_autoencoder(student_model, optimizer, criterion):
    print("Running Autoencoder Training and Evaluation...")
    
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #student_model.to(device)
    student_model.eval()  # Set to evaluation mode
    device = next(student_model.parameters()).device  # Get model device
    
    dummy_data = torch.randn(16, 1, 28, 28).to(device)  # Small batch size to reduce memory
    for epoch in range(2):
        optimizer.zero_grad()
        dummy_data_flat = dummy_data.view(dummy_data.size(0), -1)  # Flatten input
        outputs = student_model(dummy_data_flat)
        loss = criterion(outputs.view(outputs.size(0), -1), dummy_data_flat)  # Flatten outputs
        loss.backward()
        optimizer.step()
        print(f"Epoch [{epoch+1}/2], Loss: {loss.item():.4f}")
    
    return loss.item(), outputs
check_autoencoder(student_model, optimizer, criterion)